<a href="https://colab.research.google.com/github/mohanrajmit/Transfer-Learning/blob/master/CNN_bird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pathlib import Path
import numpy as np
import joblib
from tensorflow.keras.preprocessing.image import load_img,img_to_array

# Empty lists to hold the images and labels for each each image
x_train = []
y_train = []

In [2]:
!git clone https://github.com/mohanrajmit/Transfer-Learning.git

Cloning into 'Transfer-Learning'...
remote: Enumerating objects: 2235, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 2235 (delta 6), reused 0 (delta 0), pack-reused 2223
Receiving objects: 100% (2235/2235), 13.87 MiB | 19.76 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [3]:
!ls

sample_data  Transfer-Learning


In [4]:
# Load the training data set by looping over every image file
for image_file in Path("Transfer-Learning/training_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = load_img(image_file, target_size=(73, 73))

    # Convert the loaded image file to a numpy array
    image_array = img_to_array(image_data)

    # Add the current image to our list of training images
    x_train.append(image_array)

    # Add a label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_train.append(0)
    else:
        y_train.append(1)

In [5]:
# Convert the list of separate images into a single 4D numpy array. This is what Keras expects.
x_train = np.array(x_train)

y_train = np.array(y_train)





In [6]:
x_train.shape



(2000, 73, 73, 3)

In [7]:
y_train = np.array(y_train)

In [8]:
y_train.shape

(2000,)

In [9]:
#from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
#aug = ImageDataGenerator(rotation_range=30,width_shift_range=0.2,height_shift_range=0.2)

In [11]:
from tensorflow.keras.models import Sequential

In [12]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [13]:
# Empty lists to hold the images and labels for each each image
x_test = []
y_test = []

# Load the test data set by looping over every image file
for image_file in Path("Transfer-Learning/test_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = load_img(image_file, target_size=(73, 73))

    # Convert the loaded image file to a numpy array
    image_array = img_to_array(image_data)

    # Add the current image to our list of test images
    x_test.append(image_array)

    # Add an expected label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_test.append(0)
    else:
        y_test.append(1)

# Convert the list of test images to a numpy array
x_test = np.array(x_test)

y_test = np.array(y_test)

In [14]:
x_test.shape

(200, 73, 73, 3)

In [15]:
y_test.shape

(200,)

In [16]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(73,73,3),padding='same', activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [17]:
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy'])

In [18]:
model.fit(x_train,y_train,batch_size=25,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
80/80 [==============================] - 13s 19ms/step - loss: 9.4085 - accuracy: 0.5835 - val_loss: 0.6232 - val_accuracy: 0.6450
Epoch 2/10
80/80 [==============================] - 1s 17ms/step - loss: 0.5956 - accuracy: 0.6920 - val_loss: 0.5570 - val_accuracy: 0.7700
Epoch 3/10
80/80 [==============================] - 1s 14ms/step - loss: 0.5231 - accuracy: 0.7465 - val_loss: 0.6566 - val_accuracy: 0.6750
Epoch 4/10
80/80 [==============================] - 1s 13ms/step - loss: 0.4901 - accuracy: 0.7590 - val_loss: 0.5877 - val_accuracy: 0.7100
Epoch 5/10
80/80 [==============================] - 1s 13ms/step - loss: 0.4098 - accuracy: 0.8175 - val_loss: 0.6153 - val_accuracy: 0.6950
Epoch 6/10
80/80 [==============================] - 1s 12ms/step - loss: 0.3509 - accuracy: 0.8385 - val_loss: 0.6371 - val_accuracy: 0.7400
Epoch 7/10
80/80 [==============================] - 1s 13ms/step - loss: 0.2936 - accuracy: 0.8810 - val_loss: 0.8658 - val_accuracy: 0.7400
Epoch 8/10
8

In [19]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

In [20]:
# Extract features for each image (all in one pass)
predictions = model.predict(x_test)

# If the model is more than 50% sure the object is a bird, call it a bird.
# Otherwise, call it "not a bird".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

# Calculate Precision and Recall for each class
report = classification_report(y_test, predictions)
print(report)

7/7 [==============================] - 0s 19ms/step
True Positives: 77
True Negatives: 69
False Positives: 31
False Negatives: 23
              precision    recall  f1-score   support

           0       0.75      0.69      0.72       100
           1       0.71      0.77      0.74       100

    accuracy                           0.73       200
   macro avg       0.73      0.73      0.73       200
weighted avg       0.73      0.73      0.73       200

